In [1]:
!pip install pyspark
!pip install py4j
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark
from pyspark.context import SparkContext
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col 
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import min, max, mean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 60.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=69a2d0a3a8345ebb01f555a8de7116d3917d718fcc494ce266105e2ae35fe903
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
nba = files.upload()

Saving nba.csv to nba.csv


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
.appName("ALTEN")  \
.config('spark.ui.port','4050') \
.getOrCreate()

In [6]:
spark

In [11]:
df = spark.read.csv('nba.csv')

In [12]:
df.show()


+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|                 _c0|           _c1|   _c2|     _c3| _c4|   _c5|   _c6|              _c7|       _c8|
+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|                Name|          Team|Number|Position| Age|Height|Weight|          College|    Salary|
|       Avery Bradley|Boston Celtics|   0.0|      PG|25.0|   6-2| 180.0|            Texas| 7730337.0|
|         Jae Crowder|Boston Celtics|  99.0|      SF|25.0|   6-6| 235.0|        Marquette| 6796117.0|
|        John Holland|Boston Celtics|  30.0|      SG|27.0|   6-5| 205.0|Boston University|      null|
|         R.J. Hunter|Boston Celtics|  28.0|      SG|22.0|   6-5| 185.0|    Georgia State| 1148640.0|
|       Jonas Jerebko|Boston Celtics|   8.0|      PF|29.0|  6-10| 231.0|             null| 5000000.0|
|        Amir Johnson|Boston Celtics|  90.0|      PF|29.0|   6-9| 240.0|          

In [64]:
df = spark.read.format("csv").option("header","True").load("nba.csv")
df.show()

+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|                Name|          Team|Number|Position| Age|Height|Weight|          College|    Salary|
+--------------------+--------------+------+--------+----+------+------+-----------------+----------+
|       Avery Bradley|Boston Celtics|   0.0|      PG|25.0|   6-2| 180.0|            Texas| 7730337.0|
|         Jae Crowder|Boston Celtics|  99.0|      SF|25.0|   6-6| 235.0|        Marquette| 6796117.0|
|        John Holland|Boston Celtics|  30.0|      SG|27.0|   6-5| 205.0|Boston University|      null|
|         R.J. Hunter|Boston Celtics|  28.0|      SG|22.0|   6-5| 185.0|    Georgia State| 1148640.0|
|       Jonas Jerebko|Boston Celtics|   8.0|      PF|29.0|  6-10| 231.0|             null| 5000000.0|
|        Amir Johnson|Boston Celtics|  90.0|      PF|29.0|   6-9| 240.0|             null|12000000.0|
|       Jordan Mickey|Boston Celtics|  55.0|      PF|21.0|   6-8| 235.0|          

In [65]:
dfna = df.na.drop()
dfna.show()

+--------------------+--------------+------+--------+----+------+------+--------------+----------+
|                Name|          Team|Number|Position| Age|Height|Weight|       College|    Salary|
+--------------------+--------------+------+--------+----+------+------+--------------+----------+
|       Avery Bradley|Boston Celtics|   0.0|      PG|25.0|   6-2| 180.0|         Texas| 7730337.0|
|         Jae Crowder|Boston Celtics|  99.0|      SF|25.0|   6-6| 235.0|     Marquette| 6796117.0|
|         R.J. Hunter|Boston Celtics|  28.0|      SG|22.0|   6-5| 185.0| Georgia State| 1148640.0|
|       Jordan Mickey|Boston Celtics|  55.0|      PF|21.0|   6-8| 235.0|           LSU| 1170960.0|
|        Kelly Olynyk|Boston Celtics|  41.0|       C|25.0|   7-0| 238.0|       Gonzaga| 2165160.0|
|        Terry Rozier|Boston Celtics|  12.0|      PG|22.0|   6-2| 190.0|    Louisville| 1824360.0|
|        Marcus Smart|Boston Celtics|  36.0|      PG|22.0|   6-4| 220.0|Oklahoma State| 3431040.0|
|     Jare

In [69]:
df2 = dfna.filter("Number < 10").filter("Salary > 10000")
df2.show()

+-----------------+------------------+------+--------+----+------+------+--------------+----------+
|             Name|              Team|Number|Position| Age|Height|Weight|       College|    Salary|
+-----------------+------------------+------+--------+----+------+------+--------------+----------+
|    Avery Bradley|    Boston Celtics|   0.0|      PG|25.0|   6-2| 180.0|         Texas| 7730337.0|
|  Jared Sullinger|    Boston Celtics|   7.0|       C|24.0|   6-9| 260.0|    Ohio State| 2569260.0|
|    Isaiah Thomas|    Boston Celtics|   4.0|      PG|27.0|   5-9| 185.0|    Washington| 6912869.0|
|     Jarrett Jack|     Brooklyn Nets|   2.0|      PG|32.0|   6-3| 200.0|  Georgia Tech| 6300000.0|
|  Sean Kilpatrick|     Brooklyn Nets|   6.0|      SG|26.0|   6-4| 219.0|    Cincinnati|  134215.0|
|     Shane Larkin|     Brooklyn Nets|   0.0|      PG|23.0|  5-11| 175.0|    Miami (FL)| 1500000.0|
| Chris McCullough|     Brooklyn Nets|   1.0|      PF|21.0|  6-11| 200.0|      Syracuse| 1140240.0|


In [85]:
df3 = df2.filter("Age >= 25.0 ").select(['Number','Name','Position','Team','College'])
df3.show()

+------+-----------------+--------+--------------------+-------------+
|Number|             Name|Position|                Team|      College|
+------+-----------------+--------+--------------------+-------------+
|   0.0|    Avery Bradley|      PG|      Boston Celtics|        Texas|
|   4.0|    Isaiah Thomas|      PG|      Boston Celtics|   Washington|
|   2.0|     Jarrett Jack|      PG|       Brooklyn Nets| Georgia Tech|
|   6.0|  Sean Kilpatrick|      SG|       Brooklyn Nets|   Cincinnati|
|   4.0|    Arron Afflalo|      SG|     New York Knicks|         UCLA|
|   7.0|  Carmelo Anthony|      SF|     New York Knicks|     Syracuse|
|   8.0|      Robin Lopez|       C|     New York Knicks|     Stanford|
|   9.0|     Kyle O'Quinn|      PF|     New York Knicks|Norfolk State|
|   0.0|    Isaiah Canaan|      PG|  Philadelphia 76ers| Murray State|
|   7.0|      Carl Landry|      PF|  Philadelphia 76ers|       Purdue|
|   1.0|        Ish Smith|      PG|  Philadelphia 76ers|  Wake Forest|
|   5.

In [76]:
df.printSchema()
df3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- College: string (nullable = true)
 |-- Salary: string (nullable = true)

root
 |-- Number: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- College: string (nullable = true)



In [92]:
df3.orderBy(df3.Number.asc()).show()

+------+-----------------+--------+--------------------+----------------+
|Number|             Name|Position|                Team|         College|
+------+-----------------+--------+--------------------+----------------+
|   0.0|   Darrell Arthur|      PF|      Denver Nuggets|          Kansas|
|   0.0|    Avery Bradley|      PG|      Boston Celtics|           Texas|
|   0.0|    Isaiah Canaan|      PG|  Philadelphia 76ers|    Murray State|
|   0.0|    Spencer Hawes|      PF|   Charlotte Hornets|      Washington|
|   0.0|Russell Westbrook|      PG|Oklahoma City Thu...|            UCLA|
|   0.0|   Damian Lillard|      PG|Portland Trail Bl...|     Weber State|
|   0.0| Andrew Goudelock|      PG|     Houston Rockets|      Charleston|
|   0.0|       Nick Young|      SF|  Los Angeles Lakers|             USC|
|   0.0|   JaMychal Green|      PF|   Memphis Grizzlies|         Alabama|
|   0.0|     Aaron Brooks|      PG|       Chicago Bulls|          Oregon|
|   0.0|  Orlando Johnson|      SG|New

In [94]:

df4 = df3.orderBy(df3.Name.desc(),df3.Team.asc())
df4.show()
""" Its RUN but the thing is only first orderby work """

+------+-----------------+--------+--------------------+------------------+
|Number|             Name|Position|                Team|           College|
+------+-----------------+--------+--------------------+------------------+
|   5.0|      Will Barton|      SF|      Denver Nuggets|           Memphis|
|   1.0|     Tyreke Evans|      SG|New Orleans Pelicans|           Memphis|
|   1.0|     Trevor Ariza|      SF|     Houston Rockets|              UCLA|
|   9.0|       Tony Allen|      SG|   Memphis Grizzlies|    Oklahoma State|
|   2.0|      Tim Frazier|      PG|New Orleans Pelicans|        Penn State|
|   6.0|      Steve Novak|      SF|     Milwaukee Bucks|         Marquette|
|   0.0|    Spencer Hawes|      PF|   Charlotte Hornets|        Washington|
|   8.0|     Shelvin Mack|      PG|           Utah Jazz|            Butler|
|   6.0|  Sean Kilpatrick|      SG|       Brooklyn Nets|        Cincinnati|
|   4.0|       Ryan Kelly|      PF|  Los Angeles Lakers|              Duke|
|   0.0|Russ

' Its RUN but the thing is only first orderby work '

In [101]:
df4.write.csv('filter.csv')
df4.write.csv('filter2.csv')

AttributeError: ignored

In [96]:
new = spark.read.csv('filter.csv')
new.show()

+---+-----------------+---+--------------------+------------------+
|_c0|              _c1|_c2|                 _c3|               _c4|
+---+-----------------+---+--------------------+------------------+
|5.0|      Will Barton| SF|      Denver Nuggets|           Memphis|
|1.0|     Tyreke Evans| SG|New Orleans Pelicans|           Memphis|
|1.0|     Trevor Ariza| SF|     Houston Rockets|              UCLA|
|9.0|       Tony Allen| SG|   Memphis Grizzlies|    Oklahoma State|
|2.0|      Tim Frazier| PG|New Orleans Pelicans|        Penn State|
|6.0|      Steve Novak| SF|     Milwaukee Bucks|         Marquette|
|0.0|    Spencer Hawes| PF|   Charlotte Hornets|        Washington|
|8.0|     Shelvin Mack| PG|           Utah Jazz|            Butler|
|6.0|  Sean Kilpatrick| SG|       Brooklyn Nets|        Cincinnati|
|4.0|       Ryan Kelly| PF|  Los Angeles Lakers|              Duke|
|0.0|Russell Westbrook| PG|Oklahoma City Thu...|              UCLA|
|8.0|         Rudy Gay| SF|    Sacramento Kings|